# Generate Predictions Images for a Site over Time
This notebook is used to generate predictions overlays for a given location.

Outputs are saved pngs where prediction score controls the opacity of a single-color image. The script also generates RGB patches for each prediction as well.

In [ ]:
import os
import sys

import cv2
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from tensorflow import keras
from tqdm import tqdm

sys.path.append('../')
from scripts.get_s2_data_ee import get_history, get_history_polygon, get_pixel_vectors

%load_ext autoreload
%autoreload 2

In [ ]:
# Sentinel 2 band descriptions
band_descriptions = {
    'B1': 'Aerosols, 442nm',
    'B2': 'Blue, 492nm',
    'B3': 'Green, 559nm',
    'B4': 'Red, 665nm',
    'B5': 'Red Edge 1, 704nm',
    'B6': 'Red Edge 2, 739nm',
    'B7': 'Red Edge 3, 779nm',
    'B8': 'NIR, 833nm',
    'B8A': 'Red Edge 4, 864nm',
    'B9': 'Water Vapor, 943nm',
    'B11': 'SWIR 1, 1610nm',
    'B12': 'SWIR 2, 2186nm'
}

In [ ]:
# Enter rect width in degrees (0.035 max recommended) and site coordinates
rect_width = 0.0075
site_coords = [115.350242, -8.562121]
name = 'temesi'

In [ ]:
patch_history = get_history([site_coords], 
                            [name], 
                            rect_width,
                            num_months = 58,
                            start_date = '2016-03-01')

## Generate Predictions

In [ ]:
# Select model to load for predictions
model = keras.models.load_model('../models/65_mo_tpa_bootstrap_toa-12-20-2020.h5')

In [ ]:
def normalize(array):
    return np.array(array) / 3000

def predict_time_series(patch_histories, site_name, model):
    rgb_stack = []
    preds_stack = []
    dates_list = []
    
    dates = list(patch_histories.keys())
    for date in tqdm(dates):
        rgb = np.stack((patch_histories[date][site_name]['B4'],
                        patch_histories[date][site_name]['B3'],
                        patch_histories[date][site_name]['B2']), axis=-1)
        
        width, height = rgb.shape[:2]
        pixel_vectors = []
        for i in range(width):
            for j in range(height):
                pixel_vector = []
                band_lengths = [len(patch_history[date][site_name][band]) for band in band_descriptions]
                if np.array(band_lengths).all() > 0:
                    for band in band_descriptions:
                        pixel_vector.append(patch_histories[date][site_name][band][i][j])
                    pixel_vectors.append(pixel_vector)
        
        pixel_vectors = normalize(pixel_vectors)
        if len(pixel_vectors) > 0 and np.median(rgb) > 0.1:
            rgb_stack.append(normalize(rgb))
            preds = model.predict(np.expand_dims(pixel_vectors, axis=-1))
            preds_img = np.reshape(preds, (width, height, 2))[:,:,1]
            preds_stack.append(preds_img)
            dates_list.append(date)
            
    return np.array(rgb_stack), np.array(preds_stack), dates_list

In [ ]:
rgb_stack, preds_stack, dates_list = predict_time_series(patch_history, 'temesi', model)

In [ ]:
def green_blue_swap(image):
    # to play nicely with OpenCV's BGR color order
    r,g,b = cv2.split(image)
    image[:,:,0] = b
    image[:,:,1] = g
    image[:,:,2] = r
    return image

In [ ]:
def filter_small_points(image):
    # This is experimental. Filter out "hot pixel" predictions
    se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    mask = cv2.morphologyEx(image, cv2.MORPH_CLOSE, se1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
    out = image * mask
    return out

In [ ]:
def stretch_histogram(array, min_val=0.1, max_val=0.75, gamma=1.2):
    clipped = np.clip(array, min_val, max_val)
    stretched = (clipped - min_val) / (max_val - min_val) ** gamma
    return stretched

### Evaluate histogram clipping, stretching, and gamma parameters

In [ ]:
min_val = 0.1
max_val = 0.75
gamma = 1.2

original_image = rgb_stack[0]

edges, bins, patches = plt.hist(original_image.flatten(), 
                                bins=500, 
                                label='Original')

adjusted_image = stretch_histogram(original_image, min_val, max_val) ** gamma
plt.hist((adjusted_image).flatten(), 
         bins=bins, 
         alpha=0.5, 
         color='r',
         label='Adjusted')

plt.xlim([0, 1])
plt.legend()
plt.title('RGB Brightness before and after Adjustment')
plt.show()

plt.figure(dpi=150)
plt.subplot(1,2,1)
plt.imshow(np.clip(original_image, 0, 1), vmin=0, vmax=1)
plt.title('Original Image')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(adjusted_image, vmin=0, vmax=1)
plt.title('Adjusted Image')
plt.axis('off')
plt.show()

### Save Images

In [ ]:
threshold = 0.8
data_dir = os.path.join('figures/time_series/', name)
if not os.path.exists(data_dir):
    os.mkdir(data_dir)


for i, month in enumerate(dates_list):
    rgb_img = (stretch_histogram(rgb_stack[i], min_val, max_val)) ** gamma
    
    # if an image has too much cloud cover, the median rgb color will be low
    if np.median(rgb_img) > 0.1:
        bgr_img = green_blue_swap(rgb_img)
        width, height, channels = np.shape(bgr_img)
        pred_img = preds_stack[i]
        
        # points below threshold are set to black
        pred_img[pred_img < threshold] = 0
        filtered_preds = filter_small_points(pred_img)
        data = np.zeros((width, height, 4))
        
        # Create an array that is fully black. Set opacity to prediction value
        data[:,:,3] = 255 * filtered_preds
        
        cv2.imwrite(os.path.join(data_dir, f'{name}_pred_{month}_{rect_width}.png'), data)
        cv2.imwrite(os.path.join(data_dir, f'{name}_rgb_{month}_{rect_width}.png'), 255 * bgr_img)

## Generate Heatmap Contour

In [ ]:
plt.imshow(preds_stack[39])

In [ ]:
os.path.join(data_dir, 'contours', name + '_earthrise_median_contours_' + date.replace('/', '-')[:10] + '.png')

In [ ]:
plot = True

window_size = 3

areas = []
monthly_contours = []
dates = []

scale = 8
img_size = preds_stack[0].shape[0] * scale
for i in range(len(preds_stack) - window_size):
    # Create a median prediction and rgb image
    median_pred = (np.median(preds_stack[i:i+window_size], axis=0) * 255).astype(np.uint8)
    median_pred[median_pred < 0.5 * 255] = 0
    median_pred = np.array(Image.fromarray(median_pred).resize((img_size, img_size)))
    
    median_rgb = (np.median(rgb_stack[i:i+window_size], axis=0) * 255).astype(np.uint8)
    median_rgb = np.array(Image.fromarray(median_rgb).resize((img_size, img_size)))
    
    ret, thresh = cv2.threshold(median_pred, 50, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > (5 * scale) **2]
    monthly_contours.append(filtered_contours)
    
    area = np.sum([cv2.contourArea(contour) for contour in filtered_contours])
    areas.append(area)
    
    three_channel_preds = np.stack((median_pred, median_pred, median_pred), axis=-1)
    preds_contour_img = cv2.drawContours(three_channel_preds, filtered_contours, -1, (255, 0, 0), 2)
    rgb_contour_img = cv2.drawContours(median_rgb, filtered_contours, -1, (255, 0, 0), 2)
    
    cv2.imwrite(os.path.join(data_dir, 'contours', name + '_contours_filled_' + date.replace('/', '-')[:10] + '.png'), cv2.drawContours(np.zeros((img_size, img_size, 4)) * 255, filtered_contours, -1, (117, 99, 60,255), thickness=cv2.FILLED))
    
    date = dates_list[i + window_size]
    dates.append(date)
    if plot:
        plt.figure(figsize=(10,5), dpi=150)
        plt.subplot(1,2,1)
        plt.title(date + ' rgb')
        plt.imshow(rgb_contour_img, vmax=255, vmin=0)
        plt.axis('off')
        plt.subplot(1,2,2)
        plt.imshow(preds_contour_img, vmax=255, vmin=0)
        plt.title(date + ' pred')
        plt.axis('off')
        plt.show()

In [ ]:
import rasterio as rs
from rasterio import warp
x, y = warp.transform(rs.crs.CRS.from_epsg(4326), rs.crs.CRS.from_epsg(3857), [site_coords[0] - rect_width / 2, 
                                                                               site_coords[0] + rect_width / 2],                                                                      [site_coords[1] - rect_width / 2, 
                                                                               site_coords[1] + rect_width / 2])
width = abs(x[0] - x[1])
height = abs(y[0] - y[1])
area = width * height
num_pixels = img_size ** 2
pixel_area = area / num_pixels
print("m^2 per pixel:", pixel_area)

In [ ]:
plt.figure(figsize=(8,4), facecolor=(1,1,1), dpi=150)
plt.plot(np.array(areas) * pixel_area / 1000000)
plt.xticks(range(0, len(preds_stack), window_size), dates_list[0:-1:window_size], ha='right', rotation=45)
plt.title('Temesi Landfill Area over Time')
plt.ylabel('Area (km^2)')
plt.xlabel('Date')
plt.show()

In [ ]:
import geojson
import json
import datetime

write_svg = False

data_dir = os.path.join('figures/time_series/', name)

west  = site_coords[0] - rect_width / 2
east  = site_coords[0] + rect_width / 2
north = site_coords[1] + rect_width / 2
south = site_coords[1] - rect_width / 2

transform = rs.transform.from_bounds(west, south, east, north, img_size, img_size)

polygons = []

for contours, date in zip(monthly_contours, dates):
    contour_coords = []
    for contour in contours:
        for point in contour[:,0]:
            lon, lat = rs.transform.xy(transform, point[1], point[0])
            contour_coords.append([lon, lat])
        contour_coords.append(contour_coords[0])
    date = datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%Y/%m/%d %H:%M")
    polygon = geojson.Polygon(coordinates = [contour_coords])
    contour_feature = geojson.Feature(geometry=polygon, properties={'date': date})
    polygons.append(contour_feature)
    json_polygon = json.dumps(contour_feature)
    with open(os.path.join(data_dir, 'contours', name + '_median_contours_' + date.replace('/', '-')[:10] + '.geojson'), 'w') as f:
        f.write(json_polygon)
    if write_svg:
        os.system(f"svgis draw {os.path.join('/Users/ckruse/Documents/earthrise/plastics/notebooks/', data_dir, 'contours', name + '_median_contours_' + date.replace('/', '-')[:10] + '.geojson')} -o {os.path.join('/Users/ckruse/Documents/earthrise/plastics/notebooks/', data_dir, 'contours', name + '_median_contours_' + date.replace('/', '-')[:10] + '.svg')}")


json_fc = json.dumps(geojson.FeatureCollection(polygons))
with open(os.path.join(data_dir, 'contours', name + '_median_contours.geojson'), 'w') as f:
    f.write(json_fc)

In [ ]:
from keplergl import KeplerGl
contour_map = KeplerGl(height=800, data={"Contours": json_fc}, config = config)
contour_map

In [ ]:
config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [
        {
          "dataId": [
            "Contours"
          ],
          "id": "midt9x4h7",
          "name": [
            "date"
          ],
          "type": "timeRange",
          "value": [
            1470009600000,
            1484460439000
          ],
          "enlarged": True,
          "plotType": "histogram",
          "animationWindow": "free",
          "yAxis": None
        }
      ],
      "layers": [
        {
          "id": "tc2t1ik",
          "type": "geojson",
          "config": {
            "dataId": "Contours",
            "label": "Contours",
            "color": [
              227,
              26,
              26
            ],
            "columns": {
              "geojson": "_geojson"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.34,
              "strokeOpacity": 0.8,
              "thickness": 0.5,
              "strokeColor": [
                240,
                237,
                234
              ],
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": False,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": None,
            "radiusScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "Contours": [
              {
                "name": "date",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": -8.564778054477216,
      "longitude": 115.34990019544571,
      "pitch": 0,
      "zoom": 15.38240625447391,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "satellite",
      "topLayerGroups": {},
      "visibleLayerGroups": {},
      "threeDBuildingColor": [
        3.7245996603793508,
        6.518049405663864,
        13.036098811327728
      ],
      "mapStyles": {}
    }
  }
}

In [ ]:
if not os.path.exists('/Users/ckruse/Downloads/temesi_contours'):
    os.mkdir('/Users/ckruse/Downloads/temesi_contours')
    
contour_map.save_to_html(file_name='/Users/ckruse/Downloads/temesi_contours.html')

In [ ]:
areas = []
monthly_contours = []
dates = []

scale = 4
img_size = rgb_stack[0].shape[0] * scale
for prediction, rgb, date in zip(preds_stack, rgb_stack, dates_list):
    
    pred = np.copy(prediction)
    pred[pred < 0.7] = 0
    pred = Image.fromarray((pred * 255).astype(np.uint8)).resize((img_size, img_size))
    pred = np.array(pred)
    
    rgb = Image.fromarray((rgb * 255).astype(np.uint8)).resize((img_size, img_size))
    
    ret, thresh = cv2.threshold(np.copy(pred), 50, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    largest_contours = [contour for contour in sorted(contours * scale, key=cv2.contourArea)[-10:] if cv2.contourArea(contour) > 30 * scale]
    if largest_contours:
        areas.append(sum([cv2.contourArea(contour) for contour in largest_contours]))
        print([cv2.contourArea(contour) for contour in largest_contours])
        monthly_contours.append(largest_contours)
        dates.append(date)
        
    three_channel_preds = np.stack((pred, pred, pred), axis=-1)
    preds_contour_img = cv2.drawContours(three_channel_preds, largest_contours, -1, (255,0,0), 2)
    rgb_contour_img = cv2.drawContours(np.array(rgb), largest_contours, -1, (255,0,0), 2)
    
    plt.subplot(1,2,1)
    plt.title(date + ' rgb')
    plt.imshow(rgb_contour_img, vmax=255, vmin=0)
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(preds_contour_img, vmax=255, vmin=0)
    plt.title(date + ' pred')
    plt.axis('off')
    plt.show()

areas = areas[:-2]

In [ ]:
window_size = 4
mean_val = []
for i in range(len(areas) - window_size):
    mean_val.append(np.mean(areas[i:i+window_size]) * pixel_area / 1000000)

plt.figure(figsize=(8,4), facecolor=(1,1,1), dpi=150)
plt.plot(np.array(areas) * pixel_area / 1000000)
plt.plot(np.linspace(window_size / 2, len(areas) - window_size / 2, len(mean_val)), mean_val, c='r', label=f"{window_size} Month Mean")
plt.xticks(range(0, len(preds_stack), window_size), dates_list[0:-1:window_size], ha='right', rotation=45)
plt.title('Temesi Landfill Area over Time')
plt.ylabel('Area (km^2)')
plt.xlabel('Date')
plt.legend()
plt.show()


## Convert Pixel Contours to Coordinate Geojson

In [ ]:
datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%Y/%m/%d %H:%M")